### This code reads the era5 excel data and merge into a single file for later processing.
- Note the size of this file is more than 90MB

In [42]:
import pandas as pd
import os
from openpyxl import load_workbook
from tqdm import tqdm
from functools import reduce

In [43]:
# 📁 Path to your Excel folder
excel_dir = "../../era5_data_excel"

# 📂 Mapping of variable names to filenames
variable_files = {
    "temperature": "2m_temperature_6hour_2000_2025.xlsx",
    "wind_u": "10m_u_component_of_wind_6hour_2000_2025.xlsx",
    "wind_v": "10m_v_component_of_wind_6hour_2000_2025.xlsx",
    "precipitation": "total_precipitation_6hour_2000_2025.xlsx",
    "surface_runoff": "surface_runoff_6hour_2000_2025.xlsx"
}

In [44]:
# 🔁 Function to load and reshape each Excel file
def read_and_melt_variable(filepath, var_name):
    all_years = []
    wb = load_workbook(filename=filepath, read_only=True)
    for sheet in tqdm(wb.sheetnames, desc=f"Loading {var_name}"):
        df = pd.read_excel(filepath, sheet_name=sheet)
        # Reshape: latitude, longitude, datetime, value
        melted = df.melt(id_vars=["latitude", "longitude"],
                         var_name="datetime",
                         value_name=var_name)
        all_years.append(melted)
    result = pd.concat(all_years, ignore_index=True)
    return result

In [45]:
# 📥 Load all variables into long-form DataFrames
dfs = {}
for var, fname in variable_files.items():
    full_path = os.path.join(excel_dir, fname)
    dfs[var] = read_and_melt_variable(full_path, var)
    
    

Loading surface_runoff: 100%|██████████████████████████████████████████████████████████████████████████████████████████████| 26/26 [00:56<00:00,  2.18s/it]


In [46]:
# 🔗 Merge all variables on lat, lon, datetime
merged_df = reduce(lambda left, right: pd.merge(left, right, on=["latitude", "longitude", "datetime"], how="inner"), dfs.values())

# 🕒 Convert datetime from string to actual datetime
merged_df["datetime"] = pd.to_datetime(merged_df["datetime"])

# ✅ Output shape and preview
print("Merged data shape:", merged_df.shape)
merged_df.head()

Merged data shape: (5032800, 8)


,latitude,longitude,datetime,temperature,wind_u,wind_v,precipitation,surface_runoff
0,26.5,88.50,2000-01-01,8.190887,1.027008,-0.727951,0.0,0.0
1,26.5,88.75,2000-01-01,10.087372,1.219391,0.198807,0.0,0.0
2,26.5,89.00,2000-01-01,10.237762,1.422516,-0.077560,0.0,0.0
3,26.5,89.25,2000-01-01,10.550262,0.732086,-0.710373,0.0,0.0
4,26.5,89.50,2000-01-01,10.694794,-0.489594,-1.047287,0.0,0.0


In [47]:
import os

# Create folder if it doesn't exist
os.makedirs("../../data/merged_era5data", exist_ok=True)

# Save merged_df to Parquet
output_path = "../../data/merged_era5data/merged_df.parquet"
merged_df.to_parquet(output_path)
